#**Recitation Week 6 Solutions**#

#Today we will work with the “‘iris“‘ dataset built into R.



In [ ]:
library(tidyverse)
library(datasets)
head(iris)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Sepal.Length Sepal.Width Petal.Length Petal.Width Species
1 5.1          3.5         1.4          0.2         setosa 
2 4.9          3.0         1.4          0.2         setosa 
3 4.7          3.2         1.3          0.2         setosa 
4 4.6          3.1         1.5          0.2         setosa 
5 5.0          3.6         1.4          0.2         setosa 
6 5.4          3.9         1.7          0.4         setosa

#1) How many unique species of flowers are there in this dataset?

In [ ]:
unique(iris$Species)

[1] setosa     versicolor virginica 
Levels: setosa versicolor virginica

There are three species of flower.

#2) Is this data long or wide? If it is wide convert it to long, if it is long convert it to wide.

In this dataset, what would you consider one observation? I would think of an observation as one measurement. For example, the measurement in centimeters (or inches not sure which it is tbh) of one flower's petal length would be one observation. Therefore since there are columns that correspond to what should be rows, the data is in wide format. 

To format it in long format we would want to make the following changes:
1. Create a variable that uniquely identifies each flower measured (unique rows in the current dataset)
2. Keep the varialbe that indicates the species exactly as is
3. Create a variable that indicates the flower part being measured (ie it will take values "sepal" and "petal"
4. Create a variable that indicates the dimension being measured (ie it will take values "length" and "width"
5. This should yield unique numbers in the final column that correspond to unique combinations of species>flower_id>flower_part>dimension

All of this can be done at once using the ```gather()``` function from tidyverse or the ```pivot_longer()``` function.

In [ ]:
# create unique id for flower: flower_id - a number corresponding to each unique row, i'm just going to use the row number
iris_pre_tidy <- iris %>% mutate(flower_id = row_number())

#pivot_longer
iris_tidy <- iris_pre_tidy %>% 
    pivot_longer(cols = c(Sepal.Length,Sepal.Width,Petal.Length,Petal.Width), names_to = "flower_part_dimension") %>%  # pivot or "transpose"
    separate(col = flower_part_dimension, into = c("flower_part","dimension"), sep = "\\.")  # now separate into two variables

# note we needed to separate the flower part variable and the dimension variable after we pivoted the data - we can use regex!
head(iris_tidy)

Species flower_id flower_part dimension value
1 setosa  1         Sepal       Length    5.1  
2 setosa  1         Sepal       Width     3.5  
3 setosa  1         Petal       Length    1.4  
4 setosa  1         Petal       Width     0.2  
5 setosa  2         Sepal       Length    4.9  
6 setosa  2         Sepal       Width     3.0

#3) What is the frequency of each flower species? What are the values of Sepal.Length? Does it make sense to calculate the frequencies of this variable? If not how else could you look at this information?

In [ ]:
# examine frequency counts for different subsets of the data
iris_tidy %>% count(Species)  # frequency of species
iris_tidy %>% count(Species, flower_part) # frequency of Species*flower_part combinations
iris_tidy %>% count(Species, dimension) # frequency of Species*dimension combinations

Species    n  
1 setosa     200
2 versicolor 200
3 virginica  200

Species    flower_part n  
1 setosa     Petal       100
2 setosa     Sepal       100
3 versicolor Petal       100
4 versicolor Sepal       100
5 virginica  Petal       100
6 virginica  Sepal       100

Species    dimension n  
1 setosa     Length    100
2 setosa     Width     100
3 versicolor Length    100
4 versicolor Width     100
5 virginica  Length    100
6 virginica  Width     100

In [ ]:
# it is often useful to see these as percentages - this is easy to do too
iris_tidy %>% group_by(Species, flower_part) %>% summarise(freq = n(), pct = 100*freq/600)

`summarise()` regrouping output by 'Species' (override with `.groups` argument)



Species    flower_part freq pct     
1 setosa     Petal       100  16.66667
2 setosa     Sepal       100  16.66667
3 versicolor Petal       100  16.66667
4 versicolor Sepal       100  16.66667
5 virginica  Petal       100  16.66667
6 virginica  Sepal       100  16.66667

This is often very useful for data diagnostics - here were are able to confirm that we have full data since every flower species has 200 observations and every species has the same number of flowers with petal and sepal observations. Therefore we know that every flower has a measurement for length and width of sepal and petal. We also know that the data is balanced since there are the same number of flowers from each species.